# Naive Code

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_bvp

def obstacle(x,y,W1=1,r=(1,1),c=(0,0)):
    '''
    Define an area that will represent an obstacle
    
    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    ellipse = ((x - c[0])**2/r[0] + (y - c[1])**2/r[1])**20 + 1

    return W1 / ellipse

def obstacle_dx(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    x derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40* W1 * (x-c[0])*(circle)**19
    denom = r[0]*((circle)**20 + 1)**2

    return numer / denom

def obstacle_dy(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    y derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40 * W1 * (y-c[1])*(circle)**19
    denom = r[1]*((circle)**20 + 1)**2

    return numer / denom

def C(x, y, W1=1):
    '''
    Uses the obstacle function to combine all of our ellipses that we used to make our track
    '''
    return obstacle(x, y, 3*W1, r=(100,100)) - obstacle(x, y, W1,r=(10,5)) + obstacle(x, y, W1) + obstacle(x, y, W1, c=(1,0)) + obstacle(x, y, W1, c=(-1,0)) + obstacle(x, y, W1, c=(.5,0)) + obstacle(x, y, W1, c=(-.5,0))

def C_dx(x, y, W1=1):
    '''
    derivative with respect to x of the C function
    '''
    return obstacle_dx(x, y, 3*W1, r=(100,100)) - obstacle_dx(x, y, W1,r=(10,5)) + obstacle_dx(x, y, W1) + obstacle_dx(x, y, W1, c=(1,0)) + obstacle_dx(x, y, W1, c=(-1,0)) + obstacle_dx(x, y, W1, c=(.5,0)) + obstacle_dx(x, y, W1, c=(-.5,0))

def C_dy(x, y, W1=1):
    ''' 
    derivative with respect to y of the C function 
    '''
    return obstacle_dy(x, y, 3*W1, r=(100,100)) - obstacle_dy(x, y, W1,r=(10,5)) + obstacle_dy(x, y, W1) + obstacle_dy(x, y, W1, c=(1,0)) + obstacle_dy(x, y, W1, c=(-1,0)) + obstacle_dy(x, y, W1, c=(.5,0)) + obstacle_dy(x, y, W1, c=(-.5,0))

def K(delta, vx, vy, lmb = 20, cushion=.1, L = .05, M = 3.):
    '''Integral constraint due to centripetal acceleration'''
    return (cushion / (delta - np.arctan(L*M/(vx**2 + vy**2)))) ** lmb

def K_dvx(delta, vx, vy, lmb=20, cushion=.1, L=.05, M=3.):
    ''' derivative of K with respect to vx'''
    v2 = vx**2 + vy**2
    lm_v2 = L*M/(v2)
    numerator = -2*cushion*lmb*L*M*vx*(cushion/(delta-np.arctan(lm_v2)))**(lmb-1)
    denominator = (v2**2)*(lm_v2**2 + 1)*(delta - np.arctan(lm_v2))**2
    return numerator / denominator

def K_dvy(delta, vx, vy, lmb=20, cushion=.1, L=.05, M=3.):
    ''' derivative of K with respect to vy'''
    v2 = vx**2 + vy**2
    lm_v2 = L*M/(v2)
    numerator = -2*cushion*lmb*L*M*vy*(cushion/(delta-np.arctan(lm_v2)))**(lmb-1)
    denominator = (v2**2)*(lm_v2**2 + 1)*(delta - np.arctan(lm_v2))**2
    return numerator / denominator

def K_ddelta(delta, vx, vy, lmb=20, cushion=.1, L=.5, M=3.):
    ''' derivative of K with respect to delta '''
    return -lmb*(cushion / (delta - np.arctan(L*M/(vx**2 + vy**2))))**(lmb+1) / cushion

def plot_track():
    X,Y = np.meshgrid(np.linspace(-5,5,600),np.linspace(-5,5,600))
    Z = C(X,Y, W1=1)

    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(X,Y,Z,edgecolor=None,linewidth=0)
    ax.set_zlim(0,3)
    ax.view_init(elev=84,azim=90)
    ax.axis("off")
    ax.set_title("3D View")

    ax2 = fig.add_subplot(122)
    ax2.contour(X, Y, Z)
    ax2.set_xbound([-4,4])
    ax2.set_ybound([-3,3])
    ax2.set_title("2D Projection")

    ax2.axis('off')
    plt.show()

def naive_system():

    W1 = 100
    W2 = .02
    n = 200
    t = np.linspace(0, 1, n)

    C0 = C(1.25, 1.25, W1)

    def ode(t, s, p):
        return p[0] * np.array([
            s[2],
            s[3],
            1/(2*W2)*s[6],
            1/(2*W2)*s[7],
            C_dx(s[0], s[1], W1),
            C_dy(s[0], s[1], W1),
            -s[4],
            -s[5]
        ])
    
    def bc(ya, yb, p):
        return np.array([
            ya[0] + 1, ya[1] + 1.1, ya[2], ya[3], yb[0] - 1, yb[1] - 1.1, yb[2], yb[3],
            
            # H(tf) = 0
            yb[4]*yb[2] + yb[5]*yb[3] + yb[6]**2/(2*W2) + yb[7]**2/(2*W2) - (1 + C0 + (yb[6]**2+yb[7]**2/(4*W2)))
        ])
    
    guess = np.ones((8, n))

    path_x = np.concatenate((np.linspace(-1.1, 2.1, 100), 2.1*np.ones(75), np.linspace(1.1, 2.1, 25)[::-1]))
    path_y = np.concatenate((-1.1*np.ones(100), np.linspace(-1.1,1.1,75), 1.1*np.ones(25)))


    guess[0] = path_x
    guess[1] = path_y
    p0 = np.array([.5])

    sol = solve_bvp(ode, bc, t, guess, p0, max_nodes=30000)

    X = np.linspace(-4, 4, 200)
    Y = np.linspace(-3, 3, 200)
    X, Y = np.meshgrid(X, Y)
    Z = C(X, Y)

    plt.contour(X, Y, Z)
    plt.plot(guess[0], guess[1], "--k", label="Inital Guess")
    plt.scatter(-1, -1.1, marker="*", color="red", label="Start")
    plt.scatter(1, 1.1, marker="*", color="green", label="End")
    
    
    vx, vy = sol.sol(t)[2], sol.sol(t)[3]
    v = np.sqrt(vx**2 + vy ** 2)
    plt.scatter(sol.sol(t)[0], sol.sol(t)[1], marker='.',c=v, cmap="magma", label="Path of Car")
    plt.xlim([-4,4])
    plt.ylim([-3,3])
    plt.legend()
    plt.colorbar(label='Speed')  # Add colorbar to show speed
    plt.title(f"{sol.p[0]:.2F} seconds")
    plt.show()

    return sol.p[0]

naive_system()

# Grid search

In [ ]:

# L = 0.12
# M = 200
# W1 = 1
# W2 = 5
# W3 = 1
# W4 = 1
# W5 = 1
# for W1 in np.linspace(1,50,20):
#     for W2 in np.linspace(1,50,20):
#         for L in np.linspace(0.01,2,15):
#             for M in np.linspace(50,400,12):
                                
#                 def constraints_path():
#                     eps = 1e-5


#                     # ODE Function
#                     def ode(t,y,p):
#                         '''defines the ODE system''',
#                         u_acc =  1/(2*W1) * (y[8] * np.cos(y[4]) + y[9] * np.sin(y[4]))
#                         u_turn = (1/(2*W2)) * (y[11])
#                         v = np.sqrt(y[2]**2 + y[3]**2)
#                         # print(y[:,0])
#                         return p[0] * np.array([y[2],
#                                                 y[3],
#                                                 u_acc * np.cos(y[4]),
#                                                 u_acc * np.sin(y[4]),
#                                                 (v) * np.tan(y[5])/ L,
#                                                 u_turn,
#                                                 100 * C_dx(y[0], y[1]),
#                                                 100 * C_dy(y[0], y[1]),
#                                                 -y[6] - y[10] * (y[2] /v * np.tan(y[5]) / L) + K_dvx(y[5], y[2], y[3],L=L,M=M),
#                                                 -y[7] - y[10] * (y[3] /v * np.tan(y[5]) / L) + K_dvy(y[5], y[2], y[3],L=L,M=M),
#                                                 y[8] * u_acc * np.sin(y[4]) - y[9] * u_acc * np.cos(y[4]),
#                                                 (-y[10]/L) * v /((np.cos(y[5])/L)**2) + K_ddelta(y[5], y[2], y[3],L=L,M=M)])

#                     # Boundary Conditions
#                     def bc(ya, yb, p):
#                         ''' defines the boundary conditions'''
#                         u_acc =  1/(2*W1) * (yb[8] * np.cos(yb[4]) + yb[9] * np.sin(yb[4]))
#                         u_turn = (1/(2*W2)) * (yb[11])
#                         lagrangian = 1 + 100 * C(yb[0],yb[1]) + W1 * u_acc**2 + W2 * u_turn**2 + K(yb[5],yb[2],yb[3])
#                         hamiltonian_condition  = (yb[6:]).T@ np.array([yb[2],
#                                                 yb[3],
#                                                 u_acc * np.cos(yb[4]),
#                                                 u_acc * np.sin(yb[4]),
#                                                 (np.sqrt(yb[2]**2 + yb[3]**2)) * np.tan(yb[5])/ L,
#                                                 (1/(2*W2)) * (yb[11])]) -  lagrangian

#                         return np.array([ya[0] + 1, ya[1] + 1.1, ya[2], ya[3], ya[4], ya[5], yb[0] -1, yb[1] - 1.1, yb[8], yb[9], yb[10], yb[11],hamiltonian_condition])

#                                     # Define domain and initial guesses
#                     t = np.linspace(0,1,200)

#                     x0 = np.concatenate((np.linspace(-1,2.1,60), 2.1*np.ones(80), np.linspace(2.1,1,60)))
#                     y0_ = np.concatenate((-1.1*np.ones(60), np.linspace(-1.1,1.1,80), 1.1*np.ones(60)))
#                     vx0 = -(4*(np.linspace(0,1,200)-.4))**2+3
#                     vy0 = -(2*np.linspace(0,1,200)-1)**2+1
#                     theta0 = 180*np.linspace(0,1,200)**6
#                     delta0 = 750*(-(np.linspace(0,1,200)-.5)**4+.06)
                    
#                     p0 = np.ones(200)
#                     y0 = np.vstack([x0,y0_,vx0,vy0,theta0,delta0,p0,p0,p0,p0,p0,p0])
#                     p0 = np.array([2])

#                     # Solve the bvp and print optimal time
#                     sol = solve_bvp(ode, bc,t,y0,p0)
#                     # print("Optimal time", -sol.p[0]*t[-1])
#                     # if -sol.p[0]*t[-1] > 0:
                        
#                     # Create meshgrid to plot obstacle
#                     x = np.linspace(-4,4,100)
#                     y = np.linspace(-3,3,100)
#                     X, Y = np.meshgrid(x,y)
                    
#                     plt.plot(x0,y0_, "--k", label="Inital Guess")
#                     plt.contour(X, Y, C(X,Y))
#                     plt.scatter(-1, -1.1, marker="*", color="red", label="Start")
#                     plt.scatter(1, 1.1, marker="*", color="green", label="End")
#                     plt.xlabel('x(t)')
#                     plt.ylabel('y(t)')
#                     plt.title('Optimal path while avoiding collision')
                    
                    
#                     vx, vy = sol.sol(t)[2], sol.sol(t)[3]
#                     v = np.sqrt(vx**2 + vy ** 2)
#                     plt.scatter(sol.sol(t)[0], sol.sol(t)[1], marker='.',c=v, cmap="magma", label="Path of Car")
#                     plt.legend()
#                     plt.colorbar(label='Speed')  # Add colorbar to show speed
                    
#                     plt.title(f"{sol.p[0]:.2F} seconds, {W1}, {W2}, {L}, {M}")
#                     plt.show()
#                 constraints_path()
                

In [ ]:
# for W1 in np.linspace(1,60,12):
#     for W2 in np.linspace(1,60,12):
                
#         L = 0.12
#         M = 200
#         W3 = 1
#         W4 = 1
#         W5 = 1
#         def constraints_path():
#             eps = 1e-5


#             # ODE Function
#             def ode(t,y,p):
#                 '''defines the ODE system''',
#                 u_acc =  1/(2*W1) * (y[8] * np.cos(y[4]) + y[9] * np.sin(y[4]))
#                 u_turn = (1/(2*W2)) * (y[11])
#                 v = np.sqrt(y[2]**2 + y[3]**2)
#                 # print(y[:,0])
#                 return p[0] * np.array([y[2],
#                                         y[3],
#                                         u_acc * np.cos(y[4]),
#                                         u_acc * np.sin(y[4]),
#                                         (v) * np.tan(y[5])/ L,
#                                         u_turn,
#                                         100 * C_dx(y[0], y[1]),
#                                         100 * C_dy(y[0], y[1]),
#                                         -y[6] - y[10] * (y[2] /v * np.tan(y[5]) / L) + K_dvx(y[5], y[2], y[3],L=L,M=M),
#                                         -y[7] - y[10] * (y[3] /v * np.tan(y[5]) / L) + K_dvy(y[5], y[2], y[3],L=L,M=M),
#                                         y[8] * u_acc * np.sin(y[4]) - y[9] * u_acc * np.cos(y[4]),
#                                         (-y[10]/L) * v /((np.cos(y[5])/L)**2) + K_ddelta(y[5], y[2], y[3],L=L,M=M)])

#             # Boundary Conditions
#             def bc(ya, yb, p):
#                 ''' defines the boundary conditions'''
#                 u_acc =  1/(2*W1) * (yb[8] * np.cos(yb[4]) + yb[9] * np.sin(yb[4]))
#                 u_turn = (1/(2*W2)) * (yb[11])
#                 lagrangian = 1 + 100 * C(yb[0],yb[1]) + W1 * u_acc**2 + W2 * u_turn**2 + K(yb[5],yb[2],yb[3])
#                 hamiltonian_condition  = (yb[6:]).T@ np.array([yb[2],
#                                         yb[3],
#                                         u_acc * np.cos(yb[4]),
#                                         u_acc * np.sin(yb[4]),
#                                         (np.sqrt(yb[2]**2 + yb[3]**2)) * np.tan(yb[5])/ L,
#                                         (1/(2*W2)) * (yb[11])]) -  lagrangian

#                 return np.array([ya[0] + 1.25, ya[1] + 1.25, ya[2], ya[3], ya[4], ya[5], yb[0] -1.25, yb[1] - 1.25, yb[8], yb[9], yb[10], yb[11],hamiltonian_condition])

#                             # Define domain and initial guesses
#             t = np.linspace(0,1,200)

#             x0 = np.concatenate((np.linspace(-1,2.1,60), 2.1*np.ones(80), np.linspace(2.1,1,60)))
#             y0_ = np.concatenate((-1.1*np.ones(60), np.linspace(-1.1,1.1,80), 1.1*np.ones(60)))
#             vx0 = -(4*(np.linspace(0,1,200)-.4))**2+3
#             vy0 = -(2*np.linspace(0,1,200)-1)**2+1
#             theta0 = 180*np.linspace(0,1,200)**6
#             delta0 = 750*(-(np.linspace(0,1,200)-.5)**4+.06)
            
#             p0 = 2*np.ones(200)
#             y0 = np.vstack([x0,y0_,vx0,vy0,theta0,delta0,p0,p0,p0,p0,p0,p0])
#             p0 = np.array([0.5])

#             # Solve the bvp and print optimal time
#             sol = solve_bvp(ode, bc,t,y0,p0)
#             # print("Optimal time", -sol.p[0]*t[-1])
#             # if -sol.p[0]*t[-1] > 0:
                
#             # Create meshgrid to plot obstacle
#             x = np.linspace(-4,4,100)
#             y = np.linspace(-3,3,100)
#             X, Y = np.meshgrid(x,y)
            
#             plt.plot(x0,y0_, "--k", label="Inital Guess")
#             plt.contour(X, Y, C(X,Y))
#             plt.scatter(-1.25, -1.25, marker="*", color="red", label="Start")
#             plt.scatter(1.25, 1.25, marker="*", color="green", label="End")
#             plt.xlabel('x(t)')
#             plt.ylabel('y(t)')
#             plt.title('Optimal path while avoiding collision')
            
            
#             vx, vy = sol.sol(t)[2], sol.sol(t)[3]
#             v = np.sqrt(vx**2 + vy ** 2)
#             plt.scatter(sol.sol(t)[0], sol.sol(t)[1], marker='.',c=v, cmap="magma", label="Path of Car")
#             plt.legend()
#             plt.colorbar(label='Speed')  # Add colorbar to show speed
#             plt.title(f"{sol.p[0]:.2F} seconds")
#             plt.show()
#             print(W1,W2)
#         # plt.scatter(-1.25, -1.25, marker="*", color="red", label="Start")
#         # plt.scatter(1.25, 1.25, marker="*", color="green", label="End")
#         # plt.legend()
#         constraints_path()
#         # for i in np.arange(1,25,1):
#         #     for j in np.arange(1,25,1):
#         #         for k in np.arange(0.01,0.2,0.04):
                        
#         #             W1 = i
#         #             W2 = j
#         #             L = k
                        
#         #             # plt.xlim([-4,4])
#         #             # plt.ylim([-3,3])
#         #             constraints_path()

# Kinda works

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_bvp

def obstacle(x,y,W1=1,r=(1,1),c=(0,0)):
    '''
    Define an area that will represent an obstacle
    
    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    ellipse = ((x - c[0])**2/r[0] + (y - c[1])**2/r[1])**20 + 1

    return W1 / ellipse

def obstacle_dx(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    x derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40* W1 * (x-c[0])*(circle)**19
    denom = r[0]*((circle)**20 + 1)**2

    return numer / denom

def obstacle_dy(x,y,W1=1,r=(1,1,),c=(0,0)):
    '''
    y derivative of the obstacle

    Parameters:
        x (float): x position in space
        y (float): y position in space
        W1 (float): weight of cost
        r (tuple): radius in x and y direction
        c (tuple): center of the ellipse
    '''

    circle = (x - c[0])**2/r[0] + (y - c[1])**2/r[1]
    numer = -40 * W1 * (y-c[1])*(circle)**19
    denom = r[1]*((circle)**20 + 1)**2

    return numer / denom

def C(x, y, W1=1):
    '''
    Uses the obstacle function to combine all of our ellipses that we used to make our track
    '''
    return obstacle(x, y, 3*W1, r=(100,100)) - obstacle(x, y, W1,r=(10,5)) + obstacle(x, y, W1) + obstacle(x, y, W1, c=(1,0)) + obstacle(x, y, W1, c=(-1,0)) + obstacle(x, y, W1, c=(.5,0)) + obstacle(x, y, W1, c=(-.5,0))

def C_dx(x, y, W1=1):
    '''
    derivative with respect to x of the C function
    '''
    return obstacle_dx(x, y, 3*W1, r=(100,100)) - obstacle_dx(x, y, W1,r=(10,5)) + obstacle_dx(x, y, W1) + obstacle_dx(x, y, W1, c=(1,0)) + obstacle_dx(x, y, W1, c=(-1,0)) + obstacle_dx(x, y, W1, c=(.5,0)) + obstacle_dx(x, y, W1, c=(-.5,0))

def C_dy(x, y, W1=1):
    ''' 
    derivative with respect to y of the C function 
    '''
    return obstacle_dy(x, y, 3*W1, r=(100,100)) - obstacle_dy(x, y, W1,r=(10,5)) + obstacle_dy(x, y, W1) + obstacle_dy(x, y, W1, c=(1,0)) + obstacle_dy(x, y, W1, c=(-1,0)) + obstacle_dy(x, y, W1, c=(.5,0)) + obstacle_dy(x, y, W1, c=(-.5,0))

def K(delta, vx, vy, lmb = 20, cushion=.1, L = .05, M = 3.):
    '''Integral constraint due to centripetal acceleration'''
    return (cushion / (delta - np.arctan(L*M/(vx**2 + vy**2)))) ** lmb

def K_dvx(delta, vx, vy, lmb=20, cushion=.1, L=.05, M=3.):
    ''' derivative of K with respect to vx'''
    v2 = vx**2 + vy**2
    lm_v2 = L*M/(v2)
    numerator = -2*cushion*lmb*L*M*vx*(cushion/(delta-np.arctan(lm_v2)))**(lmb-1)
    denominator = (v2**2)*(lm_v2**2 + 1)*(delta - np.arctan(lm_v2))**2
    return numerator / denominator

def K_dvy(delta, vx, vy, lmb=20, cushion=.1, L=.05, M=3.):
    ''' derivative of K with respect to vy'''
    v2 = vx**2 + vy**2
    lm_v2 = L*M/(v2)
    numerator = -2*cushion*lmb*L*M*vy*(cushion/(delta-np.arctan(lm_v2)))**(lmb-1)
    denominator = (v2**2)*(lm_v2**2 + 1)*(delta - np.arctan(lm_v2))**2
    return numerator / denominator

def K_ddelta(delta, vx, vy, lmb=20, cushion=.1, L=.5, M=3.):
    ''' derivative of K with respect to delta '''
    return -lmb*(cushion / (delta - np.arctan(L*M/(vx**2 + vy**2))))**(lmb+1) / cushion

def plot_track():
    X,Y = np.meshgrid(np.linspace(-5,5,600),np.linspace(-5,5,600))
    Z = C(X,Y, W1=1)

    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(X,Y,Z,edgecolor=None,linewidth=0)
    ax.set_zlim(0,3)
    ax.view_init(elev=84,azim=90)
    ax.axis("off")
    ax.set_title("3D View")

    ax2 = fig.add_subplot(122)
    ax2.contour(X, Y, Z)
    ax2.set_xbound([-4,4])
    ax2.set_ybound([-3,3])
    ax2.set_title("2D Projection")

    ax2.axis('off')
    plt.show()

def naive_system():

    W1 = 100
    W2 = .02
    n = 200
    t = np.linspace(0, 1, n)

    C0 = C(1.25, 1.25, W1)

    def ode(t, s, p):
        return p[0] * np.array([
            s[2],
            s[3],
            1/(2*W2)*s[6],
            1/(2*W2)*s[7],
            C_dx(s[0], s[1], W1),
            C_dy(s[0], s[1], W1),
            -s[4],
            -s[5]
        ])
    
    def bc(ya, yb, p):
        return np.array([
            ya[0] + 1, ya[1] + 1.1, ya[2], ya[3], yb[0] - 1, yb[1] - 1.1, yb[2], yb[3],
            
            # H(tf) = 0
            yb[4]*yb[2] + yb[5]*yb[3] + yb[6]**2/(2*W2) + yb[7]**2/(2*W2) - (1 + C0 + (yb[6]**2+yb[7]**2/(4*W2)))
        ])
    
    guess = np.ones((8, n))

    path_x = np.concatenate((np.linspace(-1.1, 2.1, 100), 2.1*np.ones(75), np.linspace(1.1, 2.1, 25)[::-1]))
    path_y = np.concatenate((-1.1*np.ones(100), np.linspace(-1.1,1.1,75), 1.1*np.ones(25)))


    guess[0] = path_x
    guess[1] = path_y
    p0 = np.array([.5])

    sol = solve_bvp(ode, bc, t, guess, p0, max_nodes=30000)

    X = np.linspace(-4, 4, 200)
    Y = np.linspace(-3, 3, 200)
    X, Y = np.meshgrid(X, Y)
    Z = C(X, Y)


    plt.plot(guess[0], guess[1], "--k", label="Inital Guess")
    plt.contour(X, Y, Z)
    plt.plot(sol.sol(t)[0], sol.sol(t)[1], "tab:orange", label="Path of Car")
    plt.scatter(-1, -1.1, marker="*", color="red", label="Start")
    plt.scatter(1, 1.1, marker="*", color="green", label="End")
    plt.xlim([-4,4])
    plt.ylim([-3,3])
    plt.legend()
    plt.title(f"{sol.p[0]:.2F} seconds")
    plt.show()

    return sol.p[0]

naive_system()

In [ ]:
L = 0.12
M = 200
W1 = 1
W2 = 5
W3 = 1
W4 = 1
W5 = 1
def constraints_path():
    eps = 1e-5


    # ODE Function
    def ode(t,y,p):
        '''defines the ODE system''',
        u_acc =  1/(2*W1) * (y[8] * np.cos(y[4]) + y[9] * np.sin(y[4]))
        u_turn = (1/(2*W2)) * (y[11])
        v = np.sqrt(y[2]**2 + y[3]**2)
        # print(y[:,0])
        return p[0] * np.array([y[2],
                                y[3],
                                u_acc * np.cos(y[4]),
                                u_acc * np.sin(y[4]),
                                (v) * np.tan(y[5])/ L,
                                u_turn,
                                100 * C_dx(y[0], y[1]),
                                100 * C_dy(y[0], y[1]),
                                -y[6] - y[10] * (y[2] /v * np.tan(y[5]) / L) + K_dvx(y[5], y[2], y[3],L=L,M=M),
                                -y[7] - y[10] * (y[3] /v * np.tan(y[5]) / L) + K_dvy(y[5], y[2], y[3],L=L,M=M),
                                y[8] * u_acc * np.sin(y[4]) - y[9] * u_acc * np.cos(y[4]),
                                (-y[10]/L) * v /((np.cos(y[5])/L)**2) + K_ddelta(y[5], y[2], y[3],L=L,M=M)])

    # Boundary Conditions
    def bc(ya, yb, p):
        ''' defines the boundary conditions'''
        u_acc =  1/(2*W1) * (yb[8] * np.cos(yb[4]) + yb[9] * np.sin(yb[4]))
        u_turn = (1/(2*W2)) * (yb[11])
        lagrangian = 1 + 100 * C(yb[0],yb[1]) + W1 * u_acc**2 + W2 * u_turn**2 + K(yb[5],yb[2],yb[3])
        hamiltonian_condition  = (yb[6:]).T@ np.array([yb[2],
                                yb[3],
                                u_acc * np.cos(yb[4]),
                                u_acc * np.sin(yb[4]),
                                (np.sqrt(yb[2]**2 + yb[3]**2)) * np.tan(yb[5])/ L,
                                (1/(2*W2)) * (yb[11])]) -  lagrangian

        return np.array([ya[0] + 1, ya[1] + 1.1, ya[2], ya[3], ya[4], ya[5], yb[0] -1, yb[1] - 1.1, yb[8], yb[9], yb[10], yb[11],hamiltonian_condition])

                    # Define domain and initial guesses
    t = np.linspace(0,1,100)
    y0 = np.ones((12, len(t)))
    path_x = np.concatenate((np.linspace(-1.25, 2.25, 50), 2.25*np.ones(38), np.linspace(1.25, 2.25, 12)[::-1]))
    path_y = np.concatenate((-1.25*np.ones(50), np.linspace(-1.25,1.25,38), 1.25*np.ones(12)))


    y0[0] = path_x
    y0[1] = path_y
    p0 = np.array([0.5])

    # Solve the bvp and print optimal time
    sol = solve_bvp(ode, bc,t,y0,p0)
    # print("Optimal time", -sol.p[0]*t[-1])
    # if -sol.p[0]*t[-1] > 0:
        
    # Create meshgrid to plot obstacle
    x = np.linspace(-4,4,100)
    y = np.linspace(-3,3,100)
    X, Y = np.meshgrid(x,y)
    print(sol)
    print(sol.sol(t))
    # Plot optimal path and obstacle
    plt.plot(sol.y[0],sol.y[1], label = 'Optimal Path')
    
    plt.plot(path_x,path_y, "--k", label="Inital Guess")
    plt.contour(X, Y, C(X,Y))
    print(sol.sol(t))
    plt.plot(sol.sol(t)[0], sol.sol(t)[1], "tab:orange", label="Path of Car")
    plt.scatter(-1.25, -1.25, marker="*", color="red", label="Start")
    plt.scatter(1.25, 1.25, marker="*", color="green", label="End")
    plt.xlabel('x(t)')
    plt.ylabel('y(t)')
    plt.title('Optimal path while avoiding collision')
    plt.legend()
    # plt.xlim([-4,4])
    # plt.ylim([-3,3])
    plt.show()
    print(f"{sol.p[0]:.2F} seconds")
    print(W1,W2,L,M)
# plt.scatter(-1.25, -1.25, marker="*", color="red", label="Start")
# plt.scatter(1.25, 1.25, marker="*", color="green", label="End")
# plt.legend()
constraints_path()
# for i in np.arange(1,25,1):
#     for j in np.arange(1,25,1):
#         for k in np.arange(0.01,0.2,0.04):
                
#             W1 = i
#             W2 = j
#             L = k
                
#             # plt.xlim([-4,4])
#             # plt.ylim([-3,3])
#             constraints_path()